In [1]:
import pandas as pd
import torch
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
# Let's load and execute the content of the newly uploaded evaluation metric file to define the calculate_measure function
file_path = '2024OIBC-평가산식 예제[24.10.25].py'

# Reading and executing the content to load calculate_measure function
with open(file_path, 'r') as file:
    evaluation_formula_content = file.read()

# Executing the content of the file to define calculate_measure in the current environment
exec(evaluation_formula_content)

# Verifying if calculate_measure function is defined
'calculate_measure' in locals()


True

In [3]:
data = pd.read_csv('OIBC_2024_DATA/data/Cleaned_Merged_Data_Without_Index_Column.csv')


In [4]:
features = data[['실시간 임시 가격(원/kWh)', '실시간 확정 가격(원/kWh)', '하루전가격(원/kWh)', '현재 수요(kW)']]
target = data['실시간 확정 가격(원/kWh)']  # Using '실시간 확정 가격' as target for this example

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# CustomLoss를 정의해 calculate_measure를 손실로 사용
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, actual, forecast):
        # PyTorch 텐서를 numpy 배열로 변환하여 calculate_measure에 전달
        actual_np = actual.detach().cpu().numpy()
        forecast_np = forecast.detach().cpu().numpy()
        
        # calculate_measure 함수 호출
        loss_value = calculate_measure(actual_np, forecast_np)
        
        # 결과를 PyTorch 텐서로 변환하여 반환
        return torch.tensor(loss_value, requires_grad=True)

# CNNLSTMRegressor 정의
class CNNLSTMRegressor(nn.Module):
    def __init__(self, n_hidden, n_layers):
        super(CNNLSTMRegressor, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        # 1D Conv 레이어
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
        
        # LSTM 레이어
        self.lstm = nn.LSTM(
            input_size=32,  # Conv2 출력 채널에 맞춤
            hidden_size=n_hidden,
            num_layers=n_layers
            # batch_first=True
        )
        
        # Fully connected output layer
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)
    
    def forward(self, x):
        # CNN 적용
        x = self.conv1(x.view(len(x), 1, -1))  # Reshape for CNN: [batch, channel, sequence]
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        
        # LSTM 입력을 위한 차원 조정
        x = x.permute(0, 2, 1)  # [batch, sequence, features]
        
        # LSTM 적용
        lstm_out, _ = self.lstm(x)
        
        # LSTM의 마지막 타임스텝 출력 사용
        last_time_step = lstm_out[:, -1, :]
        y_pred = self.linear(last_time_step)
        return y_pred

# 데이터 전처리 및 train/test split
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))

def create_sequences(data, target, seq_len):
    sequences = []
    targets = []
    for i in range(len(data) - seq_len):
        seq = data[i:i+seq_len]
        label = target[i+seq_len-1]
        sequences.append(seq)
        targets.append(label)
    return torch.tensor(np.array(sequences), dtype=torch.float32), torch.tensor(np.array(targets), dtype=torch.float32)

seq_len = 15
X, y = create_sequences(scaled_features, scaled_target, seq_len)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 초기화 및 손실 함수/옵티마이저 설정
n_hidden = 32
n_layers = 3
model = CNNLSTMRegressor(n_hidden=n_hidden, n_layers=n_layers)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomLoss()  # CustomLoss를 손실 함수로 설정

# 모델 학습
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    y_pred = model(X_train)
    loss = criterion(y_train, y_pred)  # CustomLoss 사용
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 모델 평가
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test)
    test_loss = criterion(y_test, y_test_pred)
    print(f"Test Custom Loss: {test_loss.item()}")


Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Epoch 0, Loss: 1.4822775818828609
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9957341715312079
e1: 3.140998009371116, e2: 1.124765189424807
Accuracy: 0.9991023339317774
e1: 3.791673685643156, e2: 1.1250368423760764
Test Custom Loss: 1.609261877097715
